## <font color=Green>GMD Figure 7 </font>  
#### <font color=blue> *Extreme sea-level return-period curve* PLOT </font>


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr
#
import warnings
import matplotlib
warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)
#
colrs = 'rcbgmyrkcbgm'

#### <font color=black> *Define* Functions </font>

In [2]:
def IP_ncWF(DF,SCENARIO,EXPDATE):
    #
    modNO       = df.index.values;   # Number of datasets Module outputs
    #
    a = [];     ccomp = []
    #        
    for scenario in SCENARIO:
        for val in modNO:
            WF = df["Workflow"][val]
            # Skip if data is absent.
            if df.loc[val].at['Data_File'] == 'XXX':
               continue 
            #   
            # Pick the data files & Import nc file to dataframe. 
            dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/coupling.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
            dataFILE    = 'coupling.{arg1}.'.format(arg1=scenario) + df["Data_File"][val]
            d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
            #
            # Index for time.
            ST = 2020 ; EN = 2100
            YindST = np.where(d_nc["years"].values == ST)[0][0];   YindEN = np.where(d_nc["years"].values == EN)[0][0]
            # Save data into a new variable.
            b = d_nc.sea_level_change[:,YindST:YindEN+1,0].values
            a.append(b[None,:] )
    #stack all at once
    SAMPSLOCRISE = np.vstack(a);  SAMPSLOCRISE = np.transpose(SAMPSLOCRISE,(1,2,0))
    yrs=d_nc.years[YindST:YindEN+1].values
    return SAMPSLOCRISE, yrs; 

In [3]:
def GPDLogNExceedances(z,llambda,shape,scale,MHHW):
    #
    z0 = z
    z = np.maximum(z, 0)  #z[z<0] = 0
    #
    if shape<0:
        z=np.minimum(z,.99999*-scale/shape)
        logN = np.log(llambda*(1+(shape)*z/scale)**(-1/(shape)))
    elif shape==0:
        logN = np.log(llambda)-z/scale
    else:
        logN = np.log(llambda*(1+(shape)*z/scale)**(-1/(shape)))
    #    
    y= logN

    # (not using this since MHHW for NYC == 0.5148)
    # for those points below threshold to MHHW, put on a Gumbel.
    # if np.max(MHHW.shape)>=1:
    # np.maximum(z0, MHHW(1), z0)
    z0=np.maximum(z0, MHHW) 
    sub = np.argwhere(z0<0)
    #if np.max(MHHW.shape)>=2:
    #    MHHWfreq=MHHW(2); 
    #else:
    MHHWfreq=365.25/2  
    # y(sub) = np.log(llambda)+(np.log(MHHWfreq)-np.log(llambda))*z0(sub)/MHHW(1)
    y[tuple(sub.T)] = np.log(llambda)+(np.log(MHHWfreq)-np.log(llambda))*z0[tuple(sub.T)]/MHHW
    #
    return y    

In [4]:
sitelab='New York';
selectedSite = 12;      #PSMSL ID for NYC
threshold = 0.5148;     #GPD threshold
scale = 0.1285;         #GPD scale
shape = 0.188;          #GPD shape
llambda = 2.8085;        #Poisson Lambda

#### <font color=blue> Load Module data from </font> <mark> facts-experiments </mark>

In [5]:
# Load FACTS module names/data as a dataframe.
df = pd.read_fwf('001_IP_GMD_names_Modules_Data/mod-submod-data_WF_local_Bob_amarel_V2.txt',comment = '#')
df

,Component,Workflow,Data_File
0,total,wf1f,total.workflow.wf1f.local.nc
1,total,wf2f,total.workflow.wf2f.local.nc
2,total,wf3f,total.workflow.wf3f.local.nc
3,total,wf4,total.workflow.wf4.local.nc


In [6]:
EXPDATE     = 221217; SCENARIO    = ['ssp585']
[SAMPSLOCRISE_585, yrs] = IP_ncWF(df,SCENARIO,EXPDATE);
#
EXPDATE     = 221217; SCENARIO    = ['ssp126']
[SAMPSLOCRISE_126, yrs] = IP_ncWF(df,SCENARIO,EXPDATE);
#
#
yrs = np.insert(yrs, 0, 2010, axis=0)
#
testz = np.arange(0, 10+0.01, 0.01)
#
histcurve=np.exp(GPDLogNExceedances(testz-threshold,llambda,shape,scale,-threshold))

In [ ]:
def EFcurv(SAMPSLOCRISE,yrs,testz,threshold,llambda,shape,scale):
    #
    sampslocrise = SAMPSLOCRISE
    #
    samps=[];
    #
    samps = np.hstack((np.zeros((sampslocrise.shape[0],1)),sampslocrise)) / 1000
    #
    # samps=np.minimum(samps, np.quantile(samps, .999,axis=0)) #truncate samples viewed as physically implausible
    #
    #
    effcurve = np.empty((yrs.shape[0],testz.shape[0],)); effcurve[:] = np.nan
    #
    gg=[]; gg1=[];
    for tt in enumerate(yrs):
        # print(tt[0])
        gg  = testz[np.newaxis] - samps[:,tt[0]][np.newaxis].T
        gg1 = np.real(np.mean(np.exp(GPDLogNExceedances(gg-threshold,llambda,shape,scale,-threshold)),axis=0))
        effcurve[tt[0],:] = gg1.T
        #
    return effcurve
    #


In [25]:
def EFcurvWF(SAMPSLOCRISE,WFNO,yrs,testz,threshold,llambda,shape,scale):
    #
    workflo = np.arange(WFNO)
    for wf in workflo: 
        sampslocrise = SAMPSLOCRISE[:,:,wf]
        #
        samps=[];
        samps = np.hstack((np.zeros((sampslocrise.shape[0],1)),sampslocrise)) / 1000
        #
        # samps=np.minimum(samps, np.quantile(samps, .999,axis=0)) #truncate samples viewed as physically implausible
        #
        #
        effcurve = np.empty((yrs.shape[0],testz.shape[0],WFNO,)); effcurve[:] = np.nan
        #
        gg=[]; gg1=[];
        for tt in enumerate(yrs):
            # print(tt[0])
            gg  = testz[np.newaxis] - samps[:,tt[0]][np.newaxis].T
            gg1 = np.real(np.mean(np.exp(GPDLogNExceedances(gg-threshold,llambda,shape,scale,-threshold)),axis=0))
            effcurve[tt[0],:,wf] = np.squeeze(gg1[np.newaxis].T)
            #
    return effcurve
    #


In [15]:
sampslocrise = SAMPSLOCRISE_585[:,:,0]
samps=[];
samps = np.hstack((np.zeros((sampslocrise.shape[0],1)),sampslocrise)) / 1000
effcurve = np.empty((yrs.shape[0],testz.shape[0],)); effcurve[:] = np.nan

In [18]:
workflo = np.arange(4)
workflo.shape[0]

4

In [26]:
WFNO = 4
effcurve_ssp585 = EFcurvWF(SAMPSLOCRISE_585,WFNO,yrs,testz,threshold,llambda,shape,scale)

In [ ]:
# WF_NO = 1
# effcurve_ssp585 = EFcurv(SAMPSLOCRISE_585[:,:,WF_NO],yrs,testz,threshold,llambda,shape,scale)
# #
# effcurve_ssp126 = EFcurv(SAMPSLOCRISE_126[:,:,WF_NO],yrs,testz,threshold,llambda,shape,scale)

# <font color=green> **PLOT** </font>

In [ ]:
# Colors updated to match updated SPM colors
color_ssp119 = np.array([0, 173,207])/255
color_ssp126 = np.array([23  ,60 ,  102])/255
color_ssp245 = np.array([247 ,148,  32])/255
color_ssp370 = np.array([231 ,29 ,  37])/255
color_ssp585 = np.array([149 ,27 ,  30])/255
# scencolors=[color_SSP585, color_SSP370, color_SSP245, color_SSP126, color_SSP119]
scencolors=[color_ssp585, color_ssp245, color_ssp126]
scencolors1 = list(reversed(scencolors))
#
# Workflow Components.
#wf1e = ['GrIS-emulandice', 'AIS-emulandice', 'Glaciers-emulandice']
wf1f = ['GrIS-FittedISMIP', 'AIS-ipccar5', 'Glaciers-ipccar5-GMIP2']
#wf2e = ['GrIS-emulandice', 'AIS-larmip', 'Glaciers-emulandice']
wf2f = ['GrIS-FittedISMIP', 'AIS-larmip', 'Glaciers-ipccar5-GMIP2']
#wf3e = ['GrIS-emulandice', 'AIS-deconto21', 'Glaciers-emulandice']
wf3f = ['GrIS-FittedISMIP', 'AIS-deconto21', 'Glaciers-ipccar5-GMIP2']
wf4  = ['GrIS-bamber19', 'AIS-bamber19', 'Glaciers-ipccar5-GMIP2']
#WORKFLO = ["wf1e","wf1f","wf2e","wf2f","wf3e","wf3f","wf4"]
WORKFLO = ["wf1f","wf2f","wf3f","wf4"]
# WORKFLO = ["wf1e","wf1f","wf2e","wf2f","wf3e","wf3f"]

In [ ]:
# Set global figure size and dots per inch
plt.rcParams.update({'figure.figsize':(35,40), 'figure.dpi':100})
# Initialize the grid
grid = plt.GridSpec(4, 5, wspace=0.1, hspace=-.4)
grid00 = grid[0].subgridspec(4, 5)
grid01 = grid[1].subgridspec(4, 5)
#
# Axis Spec.
# Global
xlim  = [0,6]
ylim = [1e-3, 10]
#
XAX1 = testz
SSP  = ['ssp126','ssp585']
#
# Subplot Axis.
ax1_wf1f = plt.subplot(grid00[0, :4]); 
ax1_wf2f = plt.subplot(grid00[1, :4]); 
ax1_wf3f = plt.subplot(grid00[2, :4]); 
ax1_wf4  = plt.subplot(grid00[3, :4]); 
#
ax3_wf1f = plt.subplot(grid01[0, :4]);
ax3_wf2f = plt.subplot(grid01[1, :4]);
ax3_wf3f = plt.subplot(grid01[2, :4]);
ax3_wf4  = plt.subplot(grid01[3, :4]); 
#
#
#
for ss in SSP: # Loop through each SSP
    for wf in enumerate(WORKFLO):
        #
        effcurve_ssp585 = EFcurv(SAMPSLOCRISE_585[:,:,wf[0]],yrs,testz,threshold,llambda,shape,scale)
        effcurve_ssp126 = EFcurv(SAMPSLOCRISE_126[:,:,wf[0]],yrs,testz,threshold,llambda,shape,scale)
        #
        #
        # Select subplot axis based on workflow
        if ss == 'ssp126': 
            ax1 = eval(f'ax1_{wf[1]}');  
            effcurve = eval(f'effcurve_{ss}')
        elif ss == 'ssp585':
            ax1 = eval(f'ax3_{wf[1]}');  
            effcurve = eval(f'effcurve_{ss}')
        yylim = []
        #
        # Plot Left 
        # ax1.plot(XAX1, histcurve, label = ss, color = eval(f'color_{ss}'))
        ax1.plot(XAX1, histcurve, '-k',label = ss)
        ax1.plot(XAX1, effcurve[4,:], '-b')
        ax1.plot(XAX1, effcurve[9,:], '-r')
        ax1.set_yscale("log")
        ax1.set_ylim(ylim); ax1.set_xlim(xlim);
        ax1.set_xlabel("meters");
        ax1.set_ylabel("expected events/year");
        if wf[1] != 'wf4': ax1.xaxis.set_ticklabels([]); ax1.set_xlabel('')
        
    #

# <font color=black> **=======================================================================** </font>